In [1]:
import selenium
import requests
import bs4
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import urllib
import os

1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.


In [2]:
driver = webdriver.Chrome(r'chromedriver.exe')

In [3]:
driver.get("https://www.amazon.in/")

In [4]:
User_input = input('Enter the product you want to search : ')

Enter the product you want to search : guitars


In [5]:
search_items=driver.find_element(By.ID,'twotabsearchtextbox')
search_items.send_keys(User_input)

In [6]:
search = driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input")
search.click()

Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [7]:
product_urls = []

start=0

end=3

for page in range(start,end):

    url=driver.find_elements(By.XPATH,'//a[@class = "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')       

    for i in url:

        product_urls.append(i.get_attribute("href"))

    nexts_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')     
    nexts_button.click()
    
    time.sleep(2)



In [8]:
len(product_urls)

184

In [9]:
Brand = []   
Name = []
Rating = []
no_of_ratings = []
Price = []
Return = []
expected_delivery = []
Availability = [] 
Other_Details = []

In [10]:
for url in product_urls:
    driver.get(url)
    time.sleep(2)
    
    try:
        brand = driver.find_element(By.XPATH,'//a[@id="bylineInfo"]')      
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-')    


    time.sleep(1)
    try:
        name = driver.find_element(By.ID,'productTitle')      
        Name.append(name.text)
    except NoSuchElementException:
        Name.append('-')


    time.sleep(1)   
    try:
        rating = driver.find_element(By.XPATH,'//span[@class="a-size-base a-nowrap"]//span')  
        Rating.append(rating.text)
    except NoSuchElementException:
        Rating.append('-')


    time.sleep(1)
    try:
        no_rating = driver.find_element(By.ID,'acrCustomerReviewText')  
        no_of_ratings.append(no_rating.text)
    except NoSuchElementException:
        no_of_ratings.append('-')


    time.sleep(1)    
    try:
        price = driver.find_element(By.XPATH,'//td[@class="a-span12"]')  
        Price.append(price.text)
    except NoSuchElementException:
        Price.append('-')


    time.sleep(1)
    try:
        return_ = driver.find_element(By.XPATH,'//a[@class="a-size-small a-link-normal a-text-normal"]')  
        Return.append(return_.text)
    except NoSuchElementException:
        Return.append('-')


    time.sleep(1)
    try:
        exptd_dlvry= driver.find_element(By.XPATH,'//span[@class="a-color-error"]')  
        expected_delivery.append(exptd_dlvry.text)
    except NoSuchElementException:
         expected_delivery.append('-')


    time.sleep(1)         
    try:
        avl= driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')  
        Availability.append(avl.text)
    except NoSuchElementException:
        Availability.append('-')



    time.sleep(1)   
    try:
        othr_dtls= driver.find_element(By.XPATH,'//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')  
        Other_Details.append(othr_dtls.text)
    except NoSuchElementException:
        Other_Details.append('-')

In [11]:
Data_2 = pd.DataFrame({'Brand':Brand,'Name':Name,'Rating':Rating,'No. of ratings':no_of_ratings,'Price':Price,
                        'Return/Exchange':Return,'Expected Delivery':expected_delivery,'Availability':Availability,
                        'Other Details':Other_Details,'URL':product_urls})
Data_2

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Visit the Kadence Store,"Kadence Frontier Series, Acoustic Guitar With/...",4 out of 5,"1,585 ratings","₹4,799.00",Pay on Delivery,-,In stock.,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,4 out of 5,"1,256 ratings","₹6,899.00",Pay on Delivery,-,In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the VAULT Store,Vault ST1 Premium Electric Guitar - Metallic Blue,3.9 out of 5,33 ratings,-,Pay on Delivery,,Only 1 left in stock.,Double Action Truss Rod: Vault guitars use a d...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Brand: Photron,"Photron Acoustic Guitar, 38 inches Cutaway, PH...",3.9 out of 5,"1,534 ratings",-,Pay on Delivery,-,In stock.,"Black glossy finish, number of frets: 18, Acou...",https://www.amazon.in/gp/slredirect/picassoRed...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"16,536 ratings",-,Pay on Delivery,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
179,Brand: Moukey,Moukey 38in Left Handed Acoustic Guitar for Be...,5 out of 5,1 rating,-,7 Days Replacement,,-,Ideal for Beginner - Moukey 38” acoustic guita...,https://www.amazon.in/Moukey-Acoustic-Beginner...
180,Visit the blueberry Store,"Blueberry B40, 40 Inch Acoustic Guitar Kit inb...",4 out of 5,24 ratings,-,Pay on Delivery,-,In stock.,"Material: Top: Spruce, Side & Back: - Bass woo...",https://www.amazon.in/gp/slredirect/picassoRed...
181,Brand: LUCHILA,LUCHILA 4 String Decor Guitar Children's Music...,-,-,-,Pay on Delivery,-,In stock.,This guitar has good sound quality & good wood...,https://www.amazon.in/gp/slredirect/picassoRed...
182,Brand: LME,LME Laxmi Musical Electronics - Acoustic Picku...,3.7 out of 5,18 ratings,-,Pay on Delivery,-,In stock.,Transducer,https://www.amazon.in/gp/slredirect/picassoRed...


In [17]:
Data_2.to_csv(f"{'Amazon searched'}.csv",index=False)

Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords 'fruits','cars' and 'Machine Learning','Guitar','Cakes'.

In [2]:
user_inp1= input('Enter the product you want to search : ')
user_inp2= input('Enter the product you want to search : ')
user_inp3= input('Enter the product you want to search : ')
user_inp4= input('Enter the product you want to search : ')
user_inp5= input('Enter the product you want to search : ')

Enter the product you want to search : Fruits
Enter the product you want to search : cars
Enter the product you want to search : Machine Learing
Enter the product you want to search : Guitar
Enter the product you want to search : Cakes


In [3]:
def search_image(search): #Creating function for scraping images.
    driver=webdriver.Chrome("chromedriver.exe") 

    time.sleep(3)



# Opening the google images

    url = "https://images.google.com/"

    driver.get(url)

    search_bar = driver.find_element(By.XPATH,"//input[@class = 'gLFyf gsfi']")    # Finding the search bar using it's xpath
    
    search_bar.send_keys(search)       # Inputing "cakes" keyword to search images

    search_button = driver.find_element(By.XPATH,"//span[@class='z1asCe MZy1Rb']")    # Finding the xpath of search button

    search_button.click()        # Clicking the search button

    for _ in range(20):
    
        driver.execute_script("window.scrollBy(0,1000)")

    

        images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')



    img_urls = []

    img_data = []

    for image in images:

        source= image.get_attribute('src')

        if source is not None:

            if(source[0:4] == 'http'):

                img_urls.append(source)

            

            

    for i in range(len(img_urls)):

        if i > 10:

            break

        print("Downloading {0} of {1} images" .format(i, 10))

        response= requests.get(img_urls[i])

        file = open(r"C:\Users\prave\Videos"+str(i)+".jpg", "wb")

        file.write(response.content)

In [5]:
search_image(user_inp1)

In [6]:
search_image(user_inp2)

In [7]:
search_image(user_inp3)

In [8]:
search_image(user_inp4)

In [9]:
search_image(user_inp5)

Q4. Write a python program to search for smartphone( e.g. Oneplus Nord, pixel 4A, etc) on www.flipkart.com and scrape following details for all the search results dislayed on 1st page.Details to be scraped: 'Brand name, smartphone,colour,RAM,storage,primary camera,secondary camera, display size, battery capacity,price,product url'.Incase any detail is missing them replace it by '-'.Save your results in a dataframe and csv.

In [77]:
def flipkart(search_term):
    """Generate a URL for search Term"""
    template = 'https://www.flipkart.com/search?q={}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
    search_term = search_term.replace(' ','%20')
    return template.format(search_term)

def search_mobile(search_term):
    """Function helps to search first page from Flipkart.com"""
    
    Product_URL = []
    Brand_Name = []
    Smartphone_name = []
    Price = []
    color = []
    Display_Size = []
    Display_Resolution = []
    Processor = []
    Processor_Cores = [] 
    Internal_Storage = []
    RAM = []
    Primary_Camera = []
    Secondary_Camera = []
    Battery_Capacity = []
      
    
    driver = webdriver.Chrome()
    url = flipkart(search_term)
    driver.get(url)
    driver.maximize_window()
    
    #Scraping URL from the searched term
    for i in driver.find_elements(By.XPATH,"//div[@class='_2kHMtA']/a[@class = '_1fQZEK']"):
        Product_URL.append(i.get_attribute('href'))
        
    driver.close()
    
    
    for i in Product_URL:
        driver = webdriver.Chrome()
        driver.get(i)
        driver.maximize_window()
        
        #Scraping Smartphone_name from the searched term
        Smartphone_name.append(driver.find_element(By.XPATH,"//h1[@class = 'yhB1nd']/span[@class = 'B_NuCI']").text)
        
        #Scraping Price from the searched term
        Price.append(driver.find_element(By.XPATH,"//div[@class = '_25b18c']/div[@class = '_30jeq3 _16Jk6d']").text)
    
        btn = driver.find_element(By.XPATH,"//button[@class = '_2KpZ6l _1FH0tX']")
        btn.click()
        time.sleep(2)
        
        dummy = [] #creating two dummy list to store all the details of the Product
        for i in driver.find_elements(By.XPATH,"//div/div[@class = '_1UhVsV']"):
            dummy.append(i.text)
        
        driver.close()
        
        Dummy = [i.split('\n') for i in dummy]
        Dummy= Dummy[0]
        Dummy.remove('General')
        
        #spliting the required details from Dummy
        for i in range(0,len(Dummy)):
            if Dummy[i] == 'Color':
                color.append(Dummy[i+1])    
            if Dummy[i] == 'Display Size':
                Display_Size.append(Dummy[i+1])    
            if Dummy[i] == 'Resolution':
                Display_Resolution.append(Dummy[i+1])    
            if Dummy[i] == 'Processor Type':
                Processor.append(Dummy[i+1])      
            if Dummy[i] == 'Processor Core':
                Processor_Cores.append(Dummy[i+1])   
            if Dummy[i] == 'Internal Storage':
                Internal_Storage.append(Dummy[i+1])
            if Dummy[i] == 'RAM':
                RAM.append(Dummy[i+1])
            if Dummy[i] == 'Primary Camera':
                if Dummy[i+1] != 'Call Features':
                    Primary_Camera.append(Dummy[i+1])
            if Dummy[i] == 'Secondary Camera':
                Secondary_Camera.append(Dummy[i+1])
            if Dummy[i] == 'Battery Capacity':
                Battery_Capacity.append(Dummy[i+1])
        
    
    Brand_Name = [i.split(' ')[0] for i in Smartphone_name] #Spliting Brand details from smart Smartphone_name
    
    
    Brand_Name = Brand_Name[:18]
    Smartphone_name = Smartphone_name[:18]
    Price = Price[:18]
    color = color[:18]
    Display_Size = Display_Size[:18]
    Display_Resolution = Display_Resolution[:18]
    Processor = Processor[:18]
    Processor_Cores = Processor_Cores[:18] 
    Internal_Storage = Internal_Storage[:18]
    RAM = RAM[:18]
    Primary_Camera = Primary_Camera[:18]
    Secondary_Camera = Secondary_Camera[:18]
    Battery_Capacity = Battery_Capacity[:18]
    Product_URL = Product_URL[:18]
    
#     Creating DataFrame.
    Iteams = pd.DataFrame({"Brand_Name" :Brand_Name,
                           "Smartphone_name" :Smartphone_name,
                           "Price" :Price,
                           "Color" :color,
                           "Display_Size" :Display_Size,
                           "Display_Resolution" :Display_Resolution,
                           "Processor" :Processor,
                           "Processor_Cores" :Processor_Cores, 
                           "Internal_Storage" :Internal_Storage,
                           "RAM" :RAM,
                           "Primary_Camera" :Primary_Camera,
                           "Secondary_Camera" :Secondary_Camera,
                           "Battery_Capacity" :Battery_Capacity,
                           "Product_URL":Product_URL})
    
    #saving the dataframe in csv
    Iteams.to_csv(f"{'FlipKart searched '+search_term}.csv",index=False)
    
    return Iteams    
     

In [78]:
search_mobile("Samsung smart phones")

,Brand_Name,Smartphone_name,Price,Color,Display_Size,Display_Resolution,Processor,Processor_Cores,Internal_Storage,RAM,Primary_Camera,Secondary_Camera,Battery_Capacity,Product_URL
0,SAMSUNG,"SAMSUNG Galaxy F22 (Denim Blue, 128 GB) (6 GB...","₹13,499",Denim Blue,16.26 cm (6.4 inch),1600 x 720 Pixels,MediaTek Helio G80,Octa Core,128 GB,6 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f22-de...
1,SAMSUNG,"SAMSUNG Galaxy F22 (Denim Black, 128 GB) (6 G...","₹13,499",Denim Black,16.26 cm (6.4 inch),1600 x 720 Pixels,MediaTek Helio G80,Octa Core,128 GB,6 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f22-de...
2,SAMSUNG,"SAMSUNG Galaxy F23 5G (Aqua Blue, 128 GB) (6 ...","₹14,999",Aqua Blue,16.76 cm (6.6 inch),2408 x 1080 Pixels,Qualcomm Snapdragon 750G,Octa Core,128 GB,6 GB,50MP + 8MP + 2MP,8MP Front Camera,5000 mAh,https://www.flipkart.com/samsung-galaxy-f23-5g...
3,SAMSUNG,"SAMSUNG Galaxy F23 5G (Forest Green, 128 GB) ...","₹14,999",Forest Green,16.76 cm (6.6 inch),2408 x 1080 Pixels,Qualcomm Snapdragon 750G,Octa Core,128 GB,6 GB,50MP + 8MP + 2MP,8MP Front Camera,5000 mAh,https://www.flipkart.com/samsung-galaxy-f23-5g...
4,SAMSUNG,"SAMSUNG Galaxy F22 (Denim Black, 64 GB) (4 GB...","₹11,999",Denim Black,16.26 cm (6.4 inch),1600 x 720 Pixels,MediaTek Helio G80,Octa Core,64 GB,4 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f22-de...
5,SAMSUNG,"SAMSUNG Galaxy F22 (Denim Blue, 64 GB) (4 GB ...","₹11,999",Denim Blue,16.26 cm (6.4 inch),1600 x 720 Pixels,MediaTek Helio G80,Octa Core,64 GB,4 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f22-de...
6,SAMSUNG,"SAMSUNG Galaxy M33 5G (Deep Ocean Blue, 128 GB...","₹15,984",Deep Ocean Blue,16.76 cm (6.6 inch),1080x2400$$Pixels,Exynos 850,Octa Core,128 GB,6 GB,50MP Rear Camera,8MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-m33-5g...
7,SAMSUNG,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB) (4...","₹11,999",Waterfall Blue,16.76 cm (6.6 inch),2408 x 1080 Pixels,Exynos 850,Octa Core,64 GB,4 GB,Connectivity Features,8MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f13-wa...
8,SAMSUNG,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB) (4...","₹11,999",Nightsky Green,16.76 cm (6.6 inch),2408 x 1080 Pixels,Exynos 850,Octa Core,64 GB,4 GB,50MP + 5MP + 2MP,8MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f13-ni...
9,SAMSUNG,"SAMSUNG Galaxy F12 (Sky Blue, 64 GB) (4 GB RAM)","₹11,499",Sky Blue,16.55 cm (6.515 inch),1600 x 720 Pixels,Exynos 850,Octa Core,64 GB,4 GB,50MP + 5MP + 2MP,8MP Front Camera,6000 mAh,https://www.flipkart.com/samsung-galaxy-f12-sk...


Q5. Write a program to scrap geospatial coordinates(latitude, longitude) of a city searched on google maps.

In [24]:
driver= webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.google.co.in/maps/@25.2721149,83.0039066,15z?hl=en&authuser=0")

In [25]:
Search_loc = input("Enter Loction ")

Enter Loction Varanasi


In [26]:
Search_button = driver.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]")
Search_button.send_keys(Search_loc)
Search = driver.find_element(By.XPATH,'//button[@aria-label="Search"]')
Search.click()
coordinates = [driver.current_url]
driver.close()
coordinates = [i.split('@')[1]for i in coordinates]
coordinates = [i.split(',')[:2] for i in coordinates]
coordinates = coordinates[0]
print(f"The geospatial coordinates of {Search_loc} \n Lattitude:",coordinates[0], "\n Longitude:",coordinates[1])

The geospatial coordinates of Varanasi 
 Lattitude: 25.2721149 
 Longitude: 83.0039066


Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [35]:
driver= webdriver.Chrome()
driver.maximize_window()
driver.get("https://trak.in/india-startup-funding-investment-2015/")

In [36]:
Date = []
strtup = []
industry = []
Sub_Vertical = []
city = []
Investors = []
Investment_type = []
amount = []

for j in 54, 55,56: 
    dropdown = driver.find_element(By.XPATH,f"//div[@class = 'dataTables_length']//select[@name = 'tablepress-{j}_length']")
    dropdown.send_keys("100")
    
    
    
    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-2']"):
        Date.append(i.text)        
  
    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-3']"):
        strtup.append(i.text)

    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-4']"):
        industry.append(i.text)    
    
    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-5']"):
        Sub_Vertical.append(i.text)

    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-6']"):
        city.append(i.text)

    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-7']"):
        Investors.append(i.text)    

    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-8']"):
        Investment_type.append(i.text)

    for i in driver.find_elements(By.XPATH,f"//table[@id = 'tablepress-{j}']//td[@class = 'column-9']"):
        amount.append(i.text)
    
driver.close()
    
#Creating DataFrame from the collected details

Data_6=pd.DataFrame({"Date":Date,
                            "Start_Up":strtup,
                            "Industry":industry,
                            "Sub_Vertical":Sub_Vertical,
                            "City":city,
                            "Investors":Investors,
                            "Investment_type":Investment_type,
                            "Amount":amount})

Data_6

,Date,Start_Up,Industry,Sub_Vertical,City,Investors,Investment_type,Amount
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [117]:
driver=webdriver.Chrome()
time.sleep(3)
driver.get("https://www.digit.in/top-products/")
time.sleep(2)

In [118]:
categories= driver.find_element(By.XPATH,"//div[@class='categoty_list']/button[2]")
categories.click()

In [120]:
Best_gaming_laptop=driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[5]/div[3]/div[3]/a/div[2]/p")
Best_gaming_laptop.click()

In [121]:
product=[]
seller=[]
price=[]

#scraping the product
pro=driver.find_elements(By.XPATH,"//table[@id='summtable']/tbody/tr/td[1]")
for i in pro:
    if i.text is None :
        product.append("--") 
    else:
        product.append(i.text)

time.sleep(4)
#scraping the seller
sell=driver.find_elements(By.XPATH,"//table[@id='summtable']/tbody/tr/td[2]")
for i in sell:
    if i.text is None :
        seller.append("--") 
    else:
        seller.append(i.text)
        
time.sleep(4)        
#scraping the price 
pri=driver.find_elements(By.XPATH,"//table[@id='summtable']/tbody/tr/td[3]")
for i in pri:
    if i.text is None :
        price.append("--") 
    else:
        price.append(i.text)
# creating the dataframe from the scraped data and taking only first 10 items
df=pd.DataFrame({"product_name":product[0:10],"seller":seller[0:10],"price":price[0:10]
                })
df

,product_name,seller,price
0,MSI Titan GT77-12UHS,N/A,N/A
1,Alienware x17 R2,Dell,"₹ 389,990"
2,Acer Predator Triton 500 SE PT516-52s,N/A,"₹ 300,000"
3,Omen by HP (16-B1371TX),Amazon,"₹ 170,990"
4,Acer Predator Helios 300 AN515-45 (NH.QBRSI.0,N/A,"₹ 172,999"
5,MSI Delta 15 (A5EFK-083IN),N/A,"₹ 188,990"
6,Omen by HP (16-c0141AX),Amazon,"₹ 129,899"
7,Lenovo Legion 5i Pro (82RF00MGIN),N/A,"₹ 230,890"
8,Alienware m15 R5 Ryzen Edition icc-c780001win,Dell,"₹ 144,990"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Croma,"₹ 142,990"


Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [2]:
driver= webdriver.Chrome()

In [3]:
driver.get("https://www.forbes.com/billionaires/")

In [5]:
Person_Name = []
Person_Rank =[]
Person_Net_worth = []
Person_Age = []
Person_Citizenship = []
Person_Source = []
Person_Industry = []

Person_name = driver.find_elements(By.XPATH,'//div[@class = "personName"]')
for i in Person_name:
    try:
        Person_Name.append(i.text)
    except NoSuchElementException:
        Person_Name.append('---')
        
Person_rank = driver.find_elements(By.XPATH,'//div[@class = "rank"]')
for i in Person_rank:
    try:
        Person_Rank.append(i.text)
    except NoSuchElementException:
        Person_Rank.append('---')
        
Person_net_worth = driver.find_elements(By.XPATH,'//div[@class = "netWorth"]')
for i in Person_net_worth:
    try:
        Person_Net_worth.append(i.text)
    except NoSuchElementException:
        Person_Net_worth.append('---')
        
Person_age = driver.find_elements(By.XPATH,'//div[@class = "age"]')
for i in Person_age:
    try:
        Person_Age.append(i.text)
    except NoSuchElementException:
        Person_Age.append('---')
        

Person_citizenship = driver.find_elements(By.XPATH,"//div[@class ='countryOfCitizenship']")
for i in Person_citizenship:
    try:
        Person_Citizenship.append(i.text)
    except NoSuchElementException:
        Person_Citizenship.append('---')
        
Person_source = driver.find_elements(By.XPATH,'//div[@class = "source"]')
for i in Person_source:
    try:
        Person_Source.append(i.text)
    except NoSuchElementException:
        Person_Source.append('---')
        
Person_industry = driver.find_elements(By.XPATH,'//div[@class = "category"]')
for i in Person_industry:
    try:
        Person_Industry.append(i.text)
    except NoSuchElementException:
        Person_Industry.append('---')

In [7]:
Data_8=pd.DataFrame({
                            "Rank":Person_Rank,
                            "Name":Person_Name,
                            "Net Worth":Person_Net_worth,
                            "Age":Person_Age,
                            "Citizenship":Person_Citizenship,
                            "Source":Person_Source,
                            "Industry":Person_Industry})

Data_8

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [8]:
Data_8.to_csv(f"{'forbes'}.csv",index=False)

Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [15]:
driver=webdriver.Chrome()
time.sleep(3)

driver.get("https://www.youtube.com/watch?v=KPLWWIOCOOQ")

time.sleep(2)

In [24]:
Comment=[]

comment=driver.find_elements(By.ID,'content-text')
for i in comment:
    try:
        Comment.append(i.text)
    except NoSuchElementException:
        Comment.append("--")

print(len(Comment))


UP_Vote=[]
vote=driver.find_elements(By.XPATH,'//span[@class ="style-scope ytd-comment-action-buttons-renderer"]')    
for i in vote:
    try:
        UP_Vote.append(i.text)
    except NoSuchElementException:
        UP_Vote.append("--")
print(len(UP_Vote))

Time = []
time =driver.find_elements(By.XPATH,'//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')    
for i in time:
    try:
        Time.append(i.text)
    except NoSuchElementException:
        Time.append("--")
print(len(Time))        

540
1080
573


In [25]:
Data_9=pd.DataFrame({"Comment":Comment[:500], "UPVote":UP_Vote[:500],'Time':Time[:500]})

Data_9

,Comment,UPVote,Time
0,GOT is like a toxic ex with who reminds you ab...,,GameofThrones
1,"This is no trailer, this is a collection of hu...",8.6K,
2,"3:08 ""If you think this has a happy ending, yo...",,1 year ago
3,I miss this epic fantasy show... despite all i...,4.5K,1 year ago
4,When you realize your getting old by knowing t...,,1 year ago
...,...,...,...
495,I am really disappointed to see the end of sea...,1,1 year ago
496,I go back in time and rewrite season 8.\nMe: I...,,1 year ago
497,Good old days,,1 year ago
498,Please erase my mind so that i can watch it again,,1 year ago


Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [38]:
driver = webdriver.Chrome()
driver.get("https://www.hostelworld.com/")

In [39]:
# Location = input("Enter the location which you want to search ")
search_items=driver.find_element(By.XPATH,'//input[@id="location-text-input-field"]')
search_items.clear()
search_items.send_keys('London')

action = ActionChains(driver)
  
action.key_down(Keys.DOWN).perform()
action.key_down(Keys.ENTER).perform()

time.sleep(3)

In [40]:
search_button = driver.find_element(By.XPATH,'//button[@id="search-button"]')       # Locating search_button by xpath
search_button.click() 

In [41]:
hostel_url = []

url=driver.find_elements(By.XPATH,"//h2[@class='title title-6']//a")

for i in url:
    hostel_url.append(i.get_attribute('href'))
    
len(hostel_url)

18

In [46]:
Hostels_Name = []
description_title=[]
ratings_title=[]
total_reviews_title=[]
private_room_prices_title=[]
dorm_room_prices_title=[]

for url in hostel_url:
    driver.get(url)
    time.sleep(2)
     
    try:
        #extracting names
        hotel_name=driver.find_element(By.XPATH,'//div[@class="title-2"]')
        Hostels_Name.append(hotel_name.text)
    except NoSuchElementException:
        Hostels_Name.append("--")


    
    try:
        #extracting description
        description=driver.find_element(By.XPATH,'//div[@class="content collapse-content"]')
        description_title.append(description.text)
    except NoSuchElementException:
        description_title.append("--")
        
        
    #extracting ratings
    try:
        ratings=driver.find_element(By.XPATH,'//div[@class="score orange big"]') 
        ratings_title.append(ratings.text)
    except NoSuchElementException:
        ratings_title.append("--")

     #extracting total_reviews
    try:
        total_reviews=driver.find_element(By.XPATH,'dev[@class="keyword"]//span')
        total_reviews_title.append(total_reviews.text)
    except NoSuchElementException:
        total_reviews_title.append("--")
        
        
    #private room prices
    try:
        private_room_prices=driver.find_element(By.XPATH,'//div[@id="rate-437879-price"]')
        private_room_prices_title.append(private_room_prices.text)
    except NoSuchElementException:
        private_room_prices_title.append("--")
        
      #dorm room prices
    try:
        dorm_room_prices=driver.find_element(By.XPATH,'//div[@id="rate-437879-price"]')
        dorm_room_prices_title.append(dorm_room_prices.text)
    except NoSuchElementException:
        dorm_room_prices_title.append("--")  

In [47]:
Data_10 = pd.DataFrame({'Names':Hostels_Name,'Description':description_title,'Ratings':ratings_title,'total_reviews':total_reviews_title,'private room price':private_room_prices_title,'Dorm room price':dorm_room_prices_title})
Data_10

,Names,Description,Ratings,total_reviews,private room price,Dorm room price
0,Wander Station Varanasi,Wander Station Hostels is your home away from ...,9.6,--,--,--
1,Moustache Hostel Varanasi,Located less than 300 mts from the prestigious...,8.0,--,--,--
2,Travo tales Varanasi,"Situated in Varanasi, 1.3 km from Dasaswamedh ...",9.6,--,--,--
3,HosteLaVie - Varanasi,HosteLaVie is India’s first premium boutique h...,8.3,--,--,--
4,goStops Varanasi,Stops is a pit-stop like none other on your In...,7.1,--,--,--
5,International Travellers’ Hostel,International Travellers’ Hostel is situated i...,7.1,--,--,--
6,Backpackers Park,"Located in the heart of Varanasi, Backpacker's...",--,--,--,--
7,Sita Inn (P.G),Most professionally managed guest house locate...,--,--,--,--
8,Shiva Guest House Varanasi,Great budget accommodation. We arrived early m...,--,--,--,--
9,Good Vibes Hostel,We are a small but united community and we bel...,9.5,--,--,--


In [48]:
Data_10.to_csv(f"{'Hostels'}.csv",index=False)